In [37]:
import os
import sys
import warnings

warnings.filterwarnings("ignore")
sys.path.append('.')

In [52]:
from pathlib import Path
from datetime import datetime
import numpy as np
import pandas as pd
import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping, LearningRateMonitor
from pytorch_lightning.loggers import TensorBoardLogger
import torch

from pytorch_forecasting import Baseline, TemporalFusionTransformer, TimeSeriesDataSet
from pytorch_forecasting.data import GroupNormalizer, MultiNormalizer
from pytorch_forecasting.metrics import SMAPE, PoissonLoss, QuantileLoss, MultiLoss
from pytorch_forecasting.models.temporal_fusion_transformer.tuning import optimize_hyperparameters

import tensorflow as tf
import tensorboard as tb
tf.io.gfile = tb.compat.tensorflow_stub.io.gfile

In [39]:
sys.path.append('..')
from gen.fishs_config import fishs

fishs = list(map(str.capitalize, fishs))

In [40]:
data = pd.read_csv(r'..\time_multiple_forecasts0.csv', sep=';')
data

,time,pressure,temperature,wind,gust,wind_direction,humidity,phenomenon,uv_index,moon_direction,...,Красноперка,Налим,Густера,Амур,Ерш,Сазан,Подуст,Толстолобик,Вобла,Хариус
0,0,750,-10,1,5,ЮЗ,50,пасмурно,0,-1,...,0.105707,0.476667,0.260000,0.0,0.284375,0.020074,0.0,0.0,0.098485,0.054485
1,3,750,-10,2,9,Ю,49,пасмурно,0,-1,...,0.105707,0.476667,0.260000,0.0,0.284375,0.020074,0.0,0.0,0.098485,0.054485
2,6,750,-7,3,9,Ю,51,пасмурно,0,-1,...,0.196313,0.349556,0.482857,0.0,0.528125,0.037279,0.0,0.0,0.182900,0.101187
3,9,750,-6,4,11,ЮЗ,45,пасмурно,0,-1,...,0.181212,0.222444,0.445714,0.0,0.487500,0.034412,0.0,0.0,0.168831,0.093403
4,12,750,-1,5,13,Ю,37,пасмурно,2,-1,...,0.166111,0.222444,0.408571,0.0,0.446875,0.031544,0.0,0.0,0.154762,0.085620
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
244147,9,752,-5,0,6,С,21,пасмурно,0,1,...,0.216667,0.247722,0.367714,0.0,0.864783,0.000000,0.0,0.0,0.270130,0.152395
244148,12,753,-3,0,5,СВ,21,пасмурно,2,1,...,0.198611,0.247722,0.337071,0.0,0.792717,0.000000,0.0,0.0,0.247619,0.139695
244149,15,753,-3,0,7,С,24,пасмурно,1,1,...,0.180556,0.247722,0.306429,0.0,0.720652,0.000000,0.0,0.0,0.225108,0.126996
244150,18,751,-3,0,3,С,27,облачно,0,1,...,0.234722,0.247722,0.398357,0.0,0.936848,0.000000,0.0,0.0,0.292641,0.165095


In [41]:
def get_time_idx(row):
    try:
        return datetime(day=row['day'], month=row['month'], year=2019).timetuple().tm_yday
    except Exception:
        return -1

data['time_idx'] = data.apply(get_time_idx , axis = 1)
data = data[data['time_idx'] >= 0]
data['time_idx'] = 8 * data['time_idx'] + data["time"]
data['group'] = 'fishow'
data

,time,pressure,temperature,wind,gust,wind_direction,humidity,phenomenon,uv_index,moon_direction,...,Густера,Амур,Ерш,Сазан,Подуст,Толстолобик,Вобла,Хариус,time_idx,group
0,0,750,-10,1,5,ЮЗ,50,пасмурно,0,-1,...,0.260000,0.0,0.284375,0.020074,0.0,0.0,0.098485,0.054485,16,fishow
1,3,750,-10,2,9,Ю,49,пасмурно,0,-1,...,0.260000,0.0,0.284375,0.020074,0.0,0.0,0.098485,0.054485,19,fishow
2,6,750,-7,3,9,Ю,51,пасмурно,0,-1,...,0.482857,0.0,0.528125,0.037279,0.0,0.0,0.182900,0.101187,22,fishow
3,9,750,-6,4,11,ЮЗ,45,пасмурно,0,-1,...,0.445714,0.0,0.487500,0.034412,0.0,0.0,0.168831,0.093403,25,fishow
4,12,750,-1,5,13,Ю,37,пасмурно,2,-1,...,0.408571,0.0,0.446875,0.031544,0.0,0.0,0.154762,0.085620,28,fishow
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
244147,9,752,-5,0,6,С,21,пасмурно,0,1,...,0.367714,0.0,0.864783,0.000000,0.0,0.0,0.270130,0.152395,2921,fishow
244148,12,753,-3,0,5,СВ,21,пасмурно,2,1,...,0.337071,0.0,0.792717,0.000000,0.0,0.0,0.247619,0.139695,2924,fishow
244149,15,753,-3,0,7,С,24,пасмурно,1,1,...,0.306429,0.0,0.720652,0.000000,0.0,0.0,0.225108,0.126996,2927,fishow
244150,18,751,-3,0,3,С,27,облачно,0,1,...,0.398357,0.0,0.936848,0.000000,0.0,0.0,0.292641,0.165095,2930,fishow


In [42]:
data['month'] = data['month'].astype('str')
data['day'] = data['day'].astype('str')
data['moon_direction'] = data['moon_direction'].astype('str')
data['time'] = data['time'].astype('str')
data['uv_index'] = data['uv_index'].astype('str')

In [43]:
max_prediction_length = 8
max_encoder_length = 24
training_cutoff = data["time_idx"].max() - max_prediction_length

training = TimeSeriesDataSet(
    data[lambda x: x.time_idx <= training_cutoff],
    time_idx="time_idx",
    target=fishs,
    group_ids=["group"],
    min_encoder_length=max_encoder_length // 2,  # keep encoder length long (as it is in the validation set)
    max_encoder_length=max_encoder_length,
    min_prediction_length=1,
    max_prediction_length=max_prediction_length,
    static_categoricals=[],
    static_reals=[],
    time_varying_known_categoricals=["month", "day", "wind_direction", "phenomenon", 'moon_direction', 'time', "uv_index"],
    time_varying_known_reals=["time_idx", "pressure", "temperature", "wind", "gust", "humidity", "moon"],
    time_varying_unknown_categoricals=[],
    time_varying_unknown_reals=[],
    target_normalizer=MultiNormalizer(normalizers=[GroupNormalizer(groups=["group"], transformation="softplus") for _ in fishs]),
    add_relative_time_idx=True,
    add_target_scales=True,
    add_encoder_length=True,
    allow_missings=True
)
validation = TimeSeriesDataSet.from_dataset(training, data, predict=True, stop_randomization=True)

# create dataloaders for model
batch_size = 128  # set this between 32 to 128
train_dataloader = training.to_dataloader(train=True, batch_size=batch_size, num_workers=0)
val_dataloader = validation.to_dataloader(train=False, batch_size=batch_size * 10, num_workers=0)

In [44]:
training[0]

({'x_cat': tensor([[ 0,  0,  7,  ...,  1,  0,  0],
          [ 0,  0,  7,  ...,  0,  0,  0],
          [ 0,  0,  1,  ...,  1,  0,  0],
          ...,
          [ 0, 25,  5,  ...,  0,  6,  0],
          [ 0, 25,  5,  ...,  1,  6,  0],
          [ 0, 22,  7,  ...,  1,  2,  1]]),
  'x_cont': tensor([[162.0000,   1.0000,   1.0000,  ...,   0.7935,   0.6620, -81.5000],
          [162.0000,   1.0000,   1.0000,  ...,   0.5064,   1.2687, -81.4583],
          [162.0000,   1.0000,   1.0000,  ...,  -0.6421,   0.9316, -81.4167],
          ...,
          [162.0000,   1.0000,   1.0000,  ...,   1.8271,  -1.0909,   0.2083],
          [162.0000,   1.0000,   1.0000,  ...,   1.1380,  -1.3943,   0.2500],
          [162.0000,   1.0000,   1.0000,  ...,  -0.6421,   1.6058,   0.2917]]),
  'encoder_length': 1956,
  'decoder_length': 8,
  'encoder_target': [tensor([0.1907, 0.2007, 0.2108,  ..., 0.2237, 0.4101, 0.4287]),
   tensor([0.3449, 0.2275, 0.3229,  ..., 0.3135, 0.4225, 0.7087]),
   tensor([0.2682, 0.2414,

In [48]:
for x, (y, weight) in iter(val_dataloader):
#     print(x)
    print(y)
    print(weight)
    break

[tensor([[0.6524, 0.5592, 0.6835, 0.6835, 0.4660, 0.3728, 0.6524, 0.2342]]), tensor([[0.4293, 0.4293, 0.4651, 0.5605, 0.1789, 0.2743, 0.5247, 0.2376]]), tensor([[0.4317, 0.2767, 0.3432, 0.2657, 0.2325, 0.0775, 0.2767, 0.1013]]), tensor([[0.6073, 0.6073, 0.4357, 0.5281, 0.3169, 0.4093, 0.5149, 0.2133]]), tensor([[0.7594, 0.6632, 0.6054, 0.6054, 0.4321, 0.3359, 0.5669, 0.2431]]), tensor([[1.0552, 1.0552, 0.8658, 1.0823, 0.5952, 0.5952, 1.1634, 0.3497]]), tensor([[0.0792, 0.0939, 0.0675, 0.0822, 0.0440, 0.0147, 0.1086, 0.0442]]), tensor([[0., 0., 0., 0., 0., 0., 0., 0.]]), tensor([[0., 0., 0., 0., 0., 0., 0., 0.]]), tensor([[0.0440, 0.0528, 0.0469, 0.0293, 0.0235, 0.0088, 0.0440, 0.0284]]), tensor([[0.1254, 0.1254, 0.0528, 0.0726, 0.0528, 0.0396, 0.1056, 0.0569]]), tensor([[0., 0., 0., 0., 0., 0., 0., 0.]]), tensor([[0., 0., 0., 0., 0., 0., 0., 0.]]), tensor([[0.1878, 0.1878, 0.0966, 0.2039, 0.0429, 0.1073, 0.1878, 0.0751]]), tensor([[0., 0., 0., 0., 0., 0., 0., 0.]]), tensor([[0., 0., 0.

In [49]:
# calculate baseline mean absolute error, i.e. predict next value as the last available value from the history
for i in range(len(fishs)):
    print('fish: {}'.format(fishs[i]))
    actuals = torch.cat([y[i] for x, (y, weight) in iter(val_dataloader)])
    baseline_predictions = Baseline().predict(val_dataloader)
    print('MAE: {}'.format((actuals - baseline_predictions).abs().mean().item()))

fish: Щука


AttributeError: 'list' object has no attribute 'size'

In [54]:
pl.seed_everything(42)
trainer = pl.Trainer(
    gpus=0,
    # clipping gradients is a hyperparameter and important to prevent divergance
    # of the gradient for recurrent neural networks
    gradient_clip_val=0.1,
)


tft = TemporalFusionTransformer.from_dataset(
    training,
    # not meaningful for finding the learning rate but otherwise very important
    learning_rate=0.03,
    hidden_size=16,  # most important hyperparameter apart from learning rate
    # number of attention heads. Set to up to 4 for large datasets
    attention_head_size=1,
    dropout=0.1,  # between 0.1 and 0.3 are good values
    hidden_continuous_size=8,  # set to <= hidden_size
    output_size=[7 for _ in fishs],  # 7 quantiles by default
    loss=MultiLoss([QuantileLoss() for _ in fishs]),
    # reduce learning rate if no improvement in validation loss after x epochs
    reduce_on_plateau_patience=4,
)
print(f"Number of parameters in network: {tft.size()/1e3:.1f}k")

Global seed set to 42
GPU available: False, used: False
TPU available: None, using: 0 TPU cores


Number of parameters in network: 74.3k


In [ ]:
# find optimal learning rate
res = trainer.tuner.lr_find(
    tft,
    train_dataloader=train_dataloader,
    val_dataloaders=val_dataloader,
    max_lr=10.0,
    min_lr=1e-6,
)

print(f"suggested learning rate: {res.suggestion()}")
fig = res.plot(show=True, suggest=True)
fig.show()


   | Name                               | Type                            | Params
----------------------------------------------------------------------------------------
0  | loss                               | MultiLoss                       | 0     
1  | logging_metrics                    | ModuleList                      | 0     
2  | input_embeddings                   | MultiEmbedding                  | 1.0 K 
3  | prescalers                         | ModuleDict                      | 1.1 K 
4  | static_variable_selection          | VariableSelectionNetwork        | 41.3 K
5  | encoder_variable_selection         | VariableSelectionNetwork        | 7.2 K 
6  | decoder_variable_selection         | VariableSelectionNetwork        | 7.2 K 
7  | static_context_variable_selection  | GatedResidualNetwork            | 1.1 K 
8  | static_context_initial_hidden_lstm | GatedResidualNetwork            | 1.1 K 
9  | static_context_initial_cell_lstm   | GatedResidualNetwork            | 1.1 